# Technology preview: Femtocode

Although it was the focus of today's tutorials, Histogrammar is not an end in itself. It's part of a larger system in development to replace private skims with a query system.

The idea is that all the big data resides on a server, like MiniAOD in EOS, and instead of downloading skims for interactive analysis, you (the physicists) perform analyses directly on the central server by submitting queries and getting results as aggregations.

Of course, it has to be considerably faster than current Grid jobs. You have to get each plot back in seconds (or less) to be productive. Some early work shows that this is possible: [see talk](https://indico.cern.ch/event/630641/)).

The query system wouldn't accept C++ strings, as in our examples today, but a new query language called Femtocode. The "femto" refers to the fact that it's intended for tiny snippets. This query language is both

   * more high-level than C++, having a Pythonic syntax apart from simplified function declarations (you don't have to type "`lambda`")
   * and faster than the equivalent C++, because it has additional constraints. It's only used for querying, after all, not application development.

The example below shows what this would look like. The cell doesn't function (don't try to evaluate it), but the plot shown below it was generated from real Femtocode. (4.5x faster than objects-on-heap C++, should be 20x.)

In [ ]:
from femtocode.remote import RemoteSession

session = RemoteSession("http://doesnt.exist.yet.fnal.gov")

workflow = session.source("ZZ_13TeV_pythia8")            # pull from a named dataset
       .define(goodmuons = "muons.filter($1.pt > 5)")    # muons with pt > 5 are good
       .filter("goodmuons.size >= 2")                    # keep events with at least two
       .define(dimuon = """
           mu1, mu2 = goodmuons.maxby($1.pt, 2);         # pick the top two by pt
           energy = mu1.E + mu2.E;                       # compute imploded energy/momentum
           px = mu1.px + mu2.px;
           py = mu1.py + mu2.py;
           pz = mu1.pz + mu2.pz;

           rec(mass = sqrt(energy**2 - px**2 - py**2 - pz**2),
               pt = sqrt(px**2 + py**2),
               phi = atan2(py, px),
               eta = log((energy + pz)/(energy - pz))/2) # construct a record as output
           """)
       .bundle(                                          # make a bundle of plots (Histogrammar 2.0)
           mass = bin(120, 60, 120, "dimuon.mass"),      # using the variables we’ve made
           pt = bin(100, 0, 100, "dimuon.pt"),
           eta = bin(100, -5, 5, "dimuon.eta"),
           phi = bin(314, 0, 2*pi, "dimuon.phi + pi"),
           muons = loop("goodmuons", "mu", bundle(       # also make plots with one muon per entry
               pt = bin(100, 0, 100, "mu.pt"),
               eta = bin(100, -5, 5, "mu.eta"),
               phi = bin(314, -pi, pi, "mu.phi")
           ))
       )

pending = workflow.submit()                              # submit the query
pending["mass"].plot()                                   # and plot results while they accumulate
pending["muons"]["pt"].plot()                            # (they’ll be animated)

blocking = pending.await()                               # stop the code until the result is in

massplot = blocking.plot.root("dimuon mass [GeV]")       # convert to a familiar format, like ROOT
massplot.Draw()                                          # and use that package’s tools

![](c1.png)

For more, see the README on the [Femtocode GitHub page](https://github.com/diana-hep/femtocode).

# A real test

You can download a current snapshot to try some real tests.

In [1]:
!wget https://github.com/diana-hep/femtocode/archive/0.0.1-lpc-hats-2017.tar.gz
!tar -xzvf 0.0.1-lpc-hats-2017.tar.gz

--2017-04-24 10:27:06--  https://github.com/diana-hep/femtocode/archive/0.0.1-lpc-hats-2017.tar.gz
Resolving github.com... 192.30.253.113, 192.30.253.112
Connecting to github.com|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/diana-hep/femtocode/tar.gz/0.0.1-lpc-hats-2017 [following]
--2017-04-24 10:27:06--  https://codeload.github.com/diana-hep/femtocode/tar.gz/0.0.1-lpc-hats-2017
Resolving codeload.github.com... 192.30.253.120, 192.30.253.121
Connecting to codeload.github.com|192.30.253.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1747448 (1.7M) [application/x-gzip]
Saving to: `0.0.1-lpc-hats-2017.tar.gz'

100%[======================================>] 1,747,448   5.27M/s   in 0.3s    

2017-04-24 10:27:06 (5.27 MB/s) - `0.0.1-lpc-hats-2017.tar.gz' saved [1747448/1747448]

femtocode-0.0.1-lpc-hats-2017/
femtocode-0.0.1-lpc-hats-2017/.gitignore
femtocode-0.0.1-lpc-hats-2017/LICEN

In [2]:
%xmode plain

import os
os.chdir("femtocode-0.0.1-lpc-hats-2017/lang")

Exception reporting mode: Plain


In [3]:
from femtocode.typesystem import *
from femtocode.testdataset import TestSession
from histogrammar.tutorial import cmsdata

Below, we convert the CMS public data from objects (returned by `cmsdata.EventIterator()`) into raw arrays. The filling function takes objects and the iterator-printout displays objects, but internally, they're arrays.

In [4]:
session = TestSession()

events = session.source("ConvertedEvents",
                        met = record(px = real, py = real),
                        muons = collection(record(pt = real(0, almost(inf)),
                                                  eta = real,
                                                  phi = real(-pi, pi))),
                        jets = collection(record(mass = real(0, almost(inf)),
                                                 pt = real(0, almost(inf)),
                                                 eta = real,
                                                 phi = real(-pi, pi))))

for event in cmsdata.EventIterator():
    if events.dataset.numEntries >= 10: break
    if len(event.muons) < 2: continue
    
    # the events.dataset.types dictionary contains namedtuples that TestDataset
    # creates for our convenience in filling and printout, but not storage
    met = events.dataset.types["met"](px = event.met.px, py = event.met.py)
    
    muons = []
    jets = []
    for muon in event.muons:
        muons.append(events.dataset.types["muons[]"](
                pt = muon.pt, eta = muon.eta, phi = muon.phi))

    for jet in event.jets:
        jets.append(events.dataset.types["jets[]"](
                mass = jet.mass, pt = jet.pt, eta = jet.eta, phi = jet.phi))
    
    events.dataset.fill({"met": met,
                         "muons": muons,
                         "jets": jets})

# print them back out to be sure they were properly loaded
for event in events.dataset:
    print event

ConvertedEvents(jets=[], met=met(px=10.415139198303223, py=-3.975071907043457), muons=[muons(eta=-0.8072726430285041, phi=-1.3402297501655267, pt=30.62639541708714), muons(eta=1.167566635561553, phi=1.692420486332129, pt=27.68023248209702)])
ConvertedEvents(jets=[], met=met(px=13.8460693359375, py=-5.433445453643799), muons=[muons(eta=0.13118731817517282, phi=2.7799272554945444, pt=28.585862770896476), muons(eta=-0.022533068091126235, phi=0.9208903654472478, pt=22.591127085739817)])
ConvertedEvents(jets=[], met=met(px=-7.185929775238037, py=5.539695739746094), muons=[muons(eta=-1.3540335633673959, phi=-0.5925778147586024, pt=46.33732663932159), muons(eta=-0.7788737577631271, phi=2.616668262237698, pt=40.2181865160263)])
ConvertedEvents(jets=[], met=met(px=23.709537506103516, py=-24.167495727539062), muons=[muons(eta=1.7817394763050594, phi=2.156796212878752, pt=54.76000171755838), muons(eta=1.2107319411626294, phi=-0.9976059224065013, pt=36.05184624130502)])
ConvertedEvents(jets=[], me

In [5]:
# It's nice to look at them as though they were namedtuples, but inside, they're arrays.

for name, segment in sorted(events.dataset.groups[0].segments.items()):
    print "{0}:\n    {1}\n    {2}\n".format(name, segment.data, segment.size)

jets[]-eta:
    [-1.2853511627286673]
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

jets[]-mass:
    [10.034879849373011]
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

jets[]-phi:
    [0.8629242977205035]
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

jets[]-pt:
    [41.49617648515141]
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

met-px:
    [10.415139198303223, 13.8460693359375, -7.185929775238037, 23.709537506103516, 4.683935165405273, 13.724966049194336, -12.390504837036133, 1.8400167226791382, -10.956795692443848, -18.61488151550293]
    None

met-py:
    [-3.975071907043457, -5.433445453643799, 5.539695739746094, -24.167495727539062, 7.290733337402344, -5.720897197723389, 12.427877426147461, -0.5923038125038147, 5.798800945281982, -14.658106803894043]
    None

muons[]-eta:
    [-0.8072726430285041, 1.167566635561553, 0.13118731817517282, -0.022533068091126235, -1.3540335633673959, -0.7788737577631271, 1.7817394763050594, 1.2107319411626294, -1.910935882686489, 0.39005679149193, -2.3250265200454856, -2.276520707323011, 

Compute something familiar.

In [6]:
# Currently, the only looping capability is ".map", which can be made into a loop over
# pairs of muons only by doubly nesting it. In the Functional Playground terminology,
# this is ".table", not ".pairs", so every output is a matrix of all pairs with the
# diagonal being a muon combined with itself (whose combined mass is exactly 2*muon mass).

# Still, this gives us a chance to compute invariant masses.
# Please pardon our dust and noise.

query = events.define(mumass = "0.105658") \
              .toPython(mass = """
muons.map(mu1 => muons.map({mu2 =>

  p1x = mu1.pt * cos(mu1.phi);
  p1y = mu1.pt * sin(mu1.phi);
  p1z = mu1.pt * sinh(mu1.eta);
  E1 = sqrt(p1x**2 + p1y**2 + p1z**2 + mumass**2);

  p2x = mu2.pt * cos(mu2.phi);
  p2y = mu2.pt * sin(mu2.phi);
  p2z = mu2.pt * sinh(mu2.eta);
  E2 = sqrt(p2x**2 + p2y**2 + p2z**2 + mumass**2);

  px = p1x + p2x;
  py = p1y + p2y;
  pz = p1z + p2z;
  E = E1 + E2;

  if E**2 - px**2 - py**2 - pz**2 >= 0:
    sqrt(E**2 - px**2 - py**2 - pz**2)
  else:
    None

}))
""")

response = query.submit(debug = False)

for event in response:
    print event.mass

[[0.21131600000293618, 88.94682415422474], [88.94682415422474, 0.21131600000293618]]
[[0.21131599999910297, 40.915269530829], [40.915269530829, 0.2113160000002105]]
[[0.2113159999835684, 89.88668022494564], [89.88668022494564, 0.21131600000293618]]
[[0.21131599994913675, 92.50889324007548], [92.50889324007548, 0.21131600001369608]]
[[0.21131600000939213, 87.33152271452795], [87.33152271452795, 0.21131599999970824]]
[[0.21131600001800005, 86.73593167619306], [86.73593167619306, 0.21131600001800005]]
[[0.21131599999648026, 17.21544038748945], [17.21544038748945, 0.2113160000007842]]
[[0.21131600000455017, 90.594059891774], [90.594059891774, 0.21131600000724016]]
[[0.21131600000293618, 99.54980041733651], [99.54980041733651, 0.21131599999917022]]
[[0.21131599999648026, 90.16056408058131], [90.16056408058131, 0.2113159999835684]]


As explained above, each output is a matrix of invariant mass of all pairs of muons: the diagonal (first and last element in these 2x2s) is the invariant mass of a muon with itself: twice the muon mass. The off-diagonals are around 90 GeV, as expected.


# What just happened

You don't see it, but some rather remarkable things just happened:

   1. The Femtocode compiler examined that code block and translated the operation on objects (the high-level, physicist's view) into operations on raw arrays (for fast processing).
   2. It split the statements into a set that only need to be in a loop over muons (the `p1x` assignment through `E2` assignment) and a set that needs to be in a loop over pairs of muons (the dimuon calculation). The intermediate values `p1x` through `E2` are computed exactly once, which is not what it would look like from the fact that their definition is in the doubly nested loop.
   3. It required us to check that `E**2 - px**2 - py**2 - pz**2 >= 0` so that the square root would always be defined. Femtocode has no runtime errors: it will force you to write guarded code before submitting to the server.

**You can see all the translation into statements and generated code if you set `debug = True`.**

**You can see it complain about the square root possibly being negative if you remove the `if` and `else` parts of the last line.**

When compiled with Numba (you don't have the prerequisites to do that), the compiled Femtocode runs 4.5 times faster than the following reasonable-looking C++:

```
for (int i = 0;  i < numEvents;  i++) {
    // create muon objects from arrays
    std::vector<Muon*> muons;
    for (int j = 0;  j < numMuons[i];  j++) {
        muons.push_back(new Muon(pt[muonindex], eta[muonindex], phi[muonindex]));
        muonindex++;
    }
    
    // loop over the muon objects
    for (std::vector<Muon*>::const_iterator one = muons.begin();  one != muons.end();  ++one) {
        for (std::vector<Muon*>::const_iterator two = muons.begin();  two != muons.end();  ++two) {
            // compute dimuons
        }
    }
    
    // delete muon objects
    for (std::vector<Muon*>::const_iterator muon = muons.begin();  muon != muons.end();  ++muon) {
        delete (*muon);
    }
}
```

because of all the memory management involved in creating and destroying muon objects. Frameworks like CMSSW and ROOT create muon objects for your convenience, so that you can write C++ analysis code. The idea here is that you write Femtocode and let it translate that down into something primitive, so that it can just rip through the numerical arrays in memory. It makes HEP analysis look more like HPC (High Performance Computing; what fluid dynamics simulators do).

**From your perspective,**

   * you don't have to leave Python
   * you don't have to think about how to optimize C++, you write in a high-level functional language
   * it runs faster than it would have if you had written the C++
   * it handles all the parallelization for you, distributing the query across a large cluster
   * and if we build this right and you can get each plot in seconds/subseconds, you shouldn't have to ever skim before doing analysis.

# Call for testers!

I'm looking for some testers, possibly in the form of a focus group (like the ones we did last year). I'm primarily interested in making sure that the language is expressive enough to cover all your analysis needs. We might do the testing using a faked implementation, like the one in the "Functional Playground," so that I can quickly change it in response to your feedback.

Anyone interested? E-mail me at [pivarski@fnal.gov](mailto:pivarski@fnal.gov).

# Fun with data types

You might have noticed that the type of "`phi`" is "`real(-pi, pi)`". What are those numeric ranges about?

In [7]:
events.type("muons.map(mu => mu.phi)")

collection(real(min=-3.14159265359, max=3.14159265359))

In [8]:
events.type("muons.map(mu => mu.pt + mu.phi)")

collection(real(min=-3.14159265359, max=almost(inf)))

In [9]:
print session.source("Fake", variable = real(0.5, 1.0)).type("sin(variable)")
print session.source("Fake", variable = real(0.5, 2.0)).type("sin(variable)")
print session.source("Fake", variable = real(0.5, 10.0)).type("sin(variable)")

real(min=0.479425538604, max=0.841470984808)
real(min=0.479425538604, max=1.0)
real(min=-1.0, max=1.0)


It's so that the Femtocode compiler can _guarantee_ that there will be no errors at runtime. That lets us speed up the runtime code (no error checking) but it also lets you know about possible logic errors due to unconsidered cases. The numeric ranges are regarded as a part of the data type, so that things like division by zero are type errors.

In [10]:
session.source("Fake", x = real, y = real).type("x / y")

FemtocodeError: Function "/" does not accept arguments with the given types:

    /(real,
      real)

    Indeterminate form (0 / 0) is not allowed; constrain with if-else.

Check line:col 1:0 (pos 0):

    x / y
----^


Constrain with if-else, eh? How about this?

In [11]:
session.source("Fake", x = real, y = real).type("if y != 0: x / y else: None")

union(null, real)

That worked, but what's `union(null, real)`? It's a way of saying that the result of this calculation could either be a number (`x/y`) or a null result (`None`). Internally, that's a `NaN`, but expressing that in the data type lets the Femtocode compiler verify that you're not putting that `NaN` into a function that expects a number:

In [12]:
session.source("Fake", x = real, y = real).type("sin(if y != 0: x / y else: None)")

FemtocodeError: Function "sin" does not accept arguments with the given types:

    sin(union(null, real))

    The sin function can only be used on numbers.

Check line:col 1:0 (pos 0):

    sin(if y != 0: x / y else: None)
----^


This prevents a `NaN` from spreading through your code, infecting everything it touches, making it very difficult to figure out where the error first happened, and of course requiring a re-run over all the data. Caught as a data type, it never even _starts_ running.

Incidentally, what's the type of `y` _inside_ the if-block?

In [13]:
session.source("Fake", x = real, y = real).type("if y != 0: y else: None")

union(null, real(min=almost(-inf), max=almost(0.0)), real(min=almost(0.0), max=almost(inf)))

It's the open interval from minus infinity to zero union the open interval from zero to infinity of course. ("Almost" means an open endpoint.)

You can also have closed endpoints:

In [14]:
session.source("Fake", x = real(-inf, inf)).type("x")

extended

That's the extended reals: the set of real numbers including infinity. Some operations are happy to accept infinity as a possible argument,

In [15]:
session.source("Fake", x = real(-inf, inf)).type("x**2")

extended(min=0, max=inf)

Some are not.

In [16]:
session.source("Fake", x = real(-inf, inf)).type("sin(x)")

FemtocodeError: Function "sin" does not accept arguments with the given types:

    sin(extended)

    The sin function cannot be used on intervals that include to infinity (though almost(-inf) and almost(inf) are okay).

Check line:col 1:0 (pos 0):

    sin(x)
----^


After all, is sin(inf) 0? 1? -1? Anything in between? It doesn't converge.

Some of Femtocode's complaints might surprise you:

In [17]:
session.source("Fake", x = real(-inf, inf), y = real(-inf, inf)).type("x + y")

FemtocodeError: Function "+" does not accept arguments with the given types:

    +(extended,
      extended)

    Indeterminate form (inf + -inf, from extended real type) is not allowed; constrain with if-else.

Check line:col 1:0 (pos 0):

    x + y
----^


Of course! Infinity minus infinity is an indeterminate form, so adding two extended reals might include that possibility. That's easy to forget.

It can be constrained by letting one end of the domain not get quite infinite.

In [18]:
session.source("Fake", x = real(almost(-inf), inf), y = real(almost(-inf), inf)).type("x + y")

extended(min=almost(-inf), max=inf)

What about the `E**2 - px**2 - py**2 - pz**2 >= 0` in the dimuon example? Femtocode required this guard because the expression appears in a square root, and we're not allowed to take the square root of a negative number. But algebraically, this expression will never be less than twice the muon mass: why didn't Femtocode know?

Because Femtocode is not a symbolic algebra system. However, symbolic algebra systems like SymPy _can_ do that sort of checking, and in the future, Femtocode will use SymPy to check the algebraic meaning of the expressions, not just the way they're written. In fact, there's a lot of unnecessary work in the dimuon example: it converts the cylindrical coordinates into cartesian coordinates before computing the mass. SymPy would collapse the expression using facts like $\sin^2(\phi) + \cos^2(\phi) = 1$ and produce much less code to compute at runtime.

You can therefore focus on making the code readable, rather than fast.